# Parkinson's Disease Detection - Demonstration

This notebook demonstrates the functionality of our Parkinson's disease detection system using spiral and wave drawing tests.

## Overview

Our system implements multiple models for detecting Parkinson's disease:
1. Convolutional Neural Network (CNN)
2. Transfer Learning (VGG16, ResNet50)
3. Traditional Machine Learning with HOG features (Random Forest, SVM, Gradient Boosting)
4. Hybrid model combining CNN with clinical data

The system is designed to achieve over 85% accuracy in detecting Parkinson's disease from spiral and wave drawings.

## 1. Setting Up

In [ ]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tensorflow.keras.models import load_model
import joblib

# Import our modules
from data_loader import ParkinsonDataLoader
from model import ParkinsonsDetector
from utils import create_sample_data_structure, visualize_training_history
from config import *

# Create directories if they don't exist
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(SPIRAL_DATASET_PATH, exist_ok=True)
os.makedirs(WAVE_DATASET_PATH, exist_ok=True)
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

## 2. Preparing Sample Data

Let's create some sample data for demonstration purposes.

In [ ]:
# Create sample spiral and wave images
!python prepare_data.py --create_sample --num_samples 20 --create_clinical

## 3. Loading and Visualizing Data

In [ ]:
# Initialize data loader
data_loader = ParkinsonDataLoader()

# Load image datasets
X_spiral, y_spiral = data_loader.load_image_dataset(
    SPIRAL_DATASET_PATH, 
    image_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1])
)

X_wave, y_wave = data_loader.load_image_dataset(
    WAVE_DATASET_PATH, 
    image_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1])
)

print(f"Spiral dataset: {X_spiral.shape[0]} images, {np.sum(y_spiral)} Parkinson's, {X_spiral.shape[0] - np.sum(y_spiral)} Healthy")
print(f"Wave dataset: {X_wave.shape[0]} images, {np.sum(y_wave)} Parkinson's, {X_wave.shape[0] - np.sum(y_wave)} Healthy")

In [ ]:
# Visualize some sample images
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Show spiral images
parkinson_spiral_idx = np.where(y_spiral == 1)[0][0]
healthy_spiral_idx = np.where(y_spiral == 0)[0][0]

axes[0, 0].imshow(X_spiral[healthy_spiral_idx, :, :, 0], cmap='gray')
axes[0, 0].set_title("Healthy Spiral")
axes[0, 0].axis('off')

axes[0, 1].imshow(X_spiral[parkinson_spiral_idx, :, :, 0], cmap='gray')
axes[0, 1].set_title("Parkinson's Spiral")
axes[0, 1].axis('off')

# Show wave images
parkinson_wave_idx = np.where(y_wave == 1)[0][0]
healthy_wave_idx = np.where(y_wave == 0)[0][0]

axes[0, 2].imshow(X_wave[healthy_wave_idx, :, :, 0], cmap='gray')
axes[0, 2].set_title("Healthy Wave")
axes[0, 2].axis('off')

axes[0, 3].imshow(X_wave[parkinson_wave_idx, :, :, 0], cmap='gray')
axes[0, 3].set_title("Parkinson's Wave")
axes[0, 3].axis('off')

# Extract and visualize HOG features
X_hog_spiral = data_loader.extract_hog_features(X_spiral, params=HOG_PARAMS)
X_hog_wave = data_loader.extract_hog_features(X_wave, params=HOG_PARAMS)

# Reshape HOG features for visualization
hog_features_healthy_spiral = X_hog_spiral[healthy_spiral_idx].reshape(-1, 1)
hog_features_parkinson_spiral = X_hog_spiral[parkinson_spiral_idx].reshape(-1, 1)
hog_features_healthy_wave = X_hog_wave[healthy_wave_idx].reshape(-1, 1)
hog_features_parkinson_wave = X_hog_wave[parkinson_wave_idx].reshape(-1, 1)

axes[1, 0].plot(hog_features_healthy_spiral)
axes[1, 0].set_title("HOG Features - Healthy Spiral")

axes[1, 1].plot(hog_features_parkinson_spiral)
axes[1, 1].set_title("HOG Features - Parkinson's Spiral")

axes[1, 2].plot(hog_features_healthy_wave)
axes[1, 2].set_title("HOG Features - Healthy Wave")

axes[1, 3].plot(hog_features_parkinson_wave)
axes[1, 3].set_title("HOG Features - Parkinson's Wave")

plt.tight_layout()
plt.show()

## 4. Loading Clinical Data

In [ ]:
# Load clinical data if available
clinical_data_path = os.path.join(DATA_PATH, "parkinson_clinical_data.csv")
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print(f"Clinical data loaded: {clinical_data.shape[0]} records")
    clinical_data.head()
else:
    print("Clinical data file not found.")

## 5. Building and Training Models

Let's build and train our models.

In [ ]:
# Initialize model
model = ParkinsonsDetector(image_shape=IMAGE_SHAPE)

# Combine spiral and wave datasets
X_combined = np.concatenate([X_spiral, X_wave], axis=0)
y_combined = np.concatenate([y_spiral, y_wave], axis=0)

# Get train, validation, and test indices
train_indices, val_indices, test_indices = data_loader.get_train_val_test_indices(
    len(X_combined), val_size=0.2, test_size=0.2)

# Split the data
X_train = X_combined[train_indices]
y_train = y_combined[train_indices]
X_val = X_combined[val_indices]
y_val = y_combined[val_indices]
X_test = X_combined[test_indices]
y_test = y_combined[test_indices]

In [ ]:
# Train a small CNN model for demonstration
# In practice, we would train for more epochs
cnn_model = model.build_cnn_model()
print(cnn_model.summary())

# Train the model with fewer epochs for demonstration
demo_epochs = 5
history = model.train_cnn_model(X_train, y_train, X_val, y_val, epochs=demo_epochs)

# Visualize training history
visualize_training_history(history)

In [ ]:
# Extract HOG features for traditional ML models
X_hog_combined = data_loader.extract_hog_features(X_combined, params=HOG_PARAMS)
X_hog_train = X_hog_combined[train_indices]
X_hog_test = X_hog_combined[test_indices]

# Train Random Forest model
rf_model = model.train_traditional_ml_model(X_hog_train, y_train, model_type='rf')
print(f"Random Forest model trained with {rf_model.n_estimators} trees")

## 6. Evaluating Models

In [ ]:
# Evaluate CNN model
cnn_metrics = model.evaluate_model('cnn', cnn_model, X_test, y_test)
print("\nCNN Model Evaluation:")
for metric, value in cnn_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}:\n{value}")

In [ ]:
# Evaluate Random Forest model
rf_metrics = model.evaluate_traditional_ml_model('rf', rf_model, X_hog_test, y_test)
print("\nRandom Forest Model Evaluation:")
for metric, value in rf_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}:\n{value}")

## 7. Making Predictions

Let's see how to use our trained models to make predictions on new images.

In [ ]:
# Save the models for later use
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
cnn_model.save(os.path.join(MODEL_SAVE_PATH, "cnn_model.h5"))
joblib.dump(rf_model, os.path.join(MODEL_SAVE_PATH, "rf_model.pkl"))
print("Models saved to", MODEL_SAVE_PATH)

In [ ]:
# Import the predictor class
from predict import ParkinsonsPredictor

# Initialize predictor
predictor = ParkinsonsPredictor(model_dir=MODEL_SAVE_PATH)

# Select a test image (assuming we have saved some images)
spiral_images = os.listdir(os.path.join(SPIRAL_DATASET_PATH, "healthy"))
if spiral_images:
    test_image_path = os.path.join(SPIRAL_DATASET_PATH, "healthy", spiral_images[0])
    
    # Make a prediction
    result = predictor.predict(test_image_path, model_name="ensemble")
    
    # Visualize prediction
    fig = predictor.visualize_prediction(result)
    plt.show()
    
    # Print prediction details
    print(f"Prediction: {result['prediction']}")
    print(f"Probability: {result['probability']:.4f}")
    
    if 'individual_predictions' in result:
        print("\nIndividual model predictions:")
        for model_name, prob in result['individual_predictions'].items():
            print(f"{model_name}: {prob:.4f}")
else:
    print("No test images found. Please create sample data first.")

## 8. Launching Web Interface

The system includes a web interface for interactive prediction. Run the following command in a terminal:

```bash
python predict.py --web_interface
```

This will launch a Gradio interface where you can upload images and see predictions in real-time.

## Conclusion

This notebook demonstrated the core functionality of our Parkinson's disease detection system. The system combines multiple models (CNN, transfer learning, and traditional ML) to achieve high accuracy in detecting Parkinson's disease from spiral and wave drawing tests.

To run the full training pipeline with all models, use the following command:

```bash
python train.py --train_all
```

To evaluate all trained models, use:

```bash
python evaluate.py --plot --save_results
```